In [1]:
import sys

sys.path.append("../")

import torch
import torch.utils as utils

import pytorch_lightning as pl

from torchemlp.groups import SO, O, S, Z
from torchemlp.nn.equivariant import EMLP
from torchemlp.nn.utils import RegressionLightning, Standardize
from torchemlp.datasets import O5Synthetic

In [2]:
TRAINING_SET_SIZE = 5_000
VALIDATION_SET_SIZE = 1_000
TEST_SET_SIZE = 1_000

BATCH_SIZE = 500

N_EPOCHS = 100  # min(int(900_000 / TRAINING_SET_SIZE), 1000)

N_CHANNELS = 384
N_LAYERS = 3

DL_WORKERS = 0

In [3]:
dataset = O5Synthetic(
    TRAINING_SET_SIZE + VALIDATION_SET_SIZE + TEST_SET_SIZE, device="cuda"
)
f"Input type: {dataset.repin(dataset.G)}, output type: {dataset.repout(dataset.G)}"

'Input type: 2V, output type: V⁰'

In [4]:
split_data = utils.data.random_split(
    dataset, [TRAINING_SET_SIZE, VALIDATION_SET_SIZE, TEST_SET_SIZE]
)

train_loader = utils.data.DataLoader(
    split_data[0], batch_size=BATCH_SIZE, num_workers=DL_WORKERS, shuffle=True
)
val_loader = utils.data.DataLoader(
    split_data[1], batch_size=BATCH_SIZE, num_workers=DL_WORKERS
)
test_loader = utils.data.DataLoader(
    split_data[2], batch_size=BATCH_SIZE, num_workers=DL_WORKERS
)

In [5]:
model = Standardize(
    EMLP(dataset.repin, dataset.repout, dataset.G, N_CHANNELS, N_LAYERS), dataset.stats
).cuda()
plmodel = RegressionLightning(model)

In [6]:
trainer = pl.Trainer(
    limit_train_batches=BATCH_SIZE, max_epochs=N_EPOCHS, accelerator="gpu"
)
trainer.fit(plmodel, train_loader, val_loader)
trainer.test(plmodel, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/eli/anaconda3/envs/l2e10/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type        | Params
--------------------------------------
0 | model | Standardize | 480 K 
--------------------------------------
480 K     Trainable params
0         Non-trainable params
480 K     Total params
1

Sanity Checking: 0it [00:00, ?it/s]

/home/eli/anaconda3/envs/l2e10/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "/home/eli/Documents/2023/Ryan/torchemlp/examples/../torchemlp/reps/reps_base.py", line 1398, in helper
    @torch.jit.script
    def helper(to_container: torch.Tensor, params: torch.Tensor, i: int, i_end: int, W_mult: int, x: torch.Tensor, bids: torch.Tensor, rep_size: int, n: int, bs: int, local_w_size: int):
        bilinear_params = params[i:i_end].reshape(W_mult, n)
                          ~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE
        to_container = (bilinear_params @ x[:, bids].T.reshape(n, rep_size * bs)).reshape(local_w_size,bs).T
        return to_container
RuntimeError: shape '[8200, 5]' is invalid for input of size 0
